In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

np.random.seed(45)
num_samples = 40
    
# Generate data
x1 = np.random.uniform(-1, 1, num_samples)
f_x = 3*x1 + 4
eps = np.random.randn(num_samples)
y = f_x + eps

In [2]:
def reg(x,theta_0,theta_1):
    return theta_1*x + theta_0
def mse(y_hat,y):
    return torch.mean((y_hat-y)**2)

In [3]:
x1_torch = torch.from_numpy(x1)
y_torch = torch.from_numpy(y)
theta_0 = torch.tensor(1.0,requires_grad=True)
theta_1 = torch.tensor(0.0, requires_grad=True)

y_hat = reg(x1_torch,theta_0,theta_1)
loss = mse(y_hat,y_torch)
loss.backward()
theta_0_true_grad = theta_0.grad.item()
theta_1_true_grad = theta_1.grad.item()
print('The True gradient values of theta_0, theta_1:',theta_0_true_grad,theta_1_true_grad)

The True gradient values of theta_0, theta_1: -5.447053909301758 -1.1946825981140137


In [4]:
t0_stochastic_gradients = []
t1_stochastic_gradients = []
for i in range(num_samples):
    theta_0.grad = None
    theta_1.grad = None
    y_hat = reg(x1_torch[i],theta_0,theta_1)
    loss = mse(y_hat,y_torch[i])
    loss.backward()
    t0_stochastic_gradients.append(theta_0.grad.item())
    t1_stochastic_gradients.append(theta_1.grad.item())

t0_avg_grad = sum(t0_stochastic_gradients)/num_samples
t1_avg_grad = sum(t1_stochastic_gradients)/num_samples
print('The True gradient values of theta_0, theta_1:',theta_0_true_grad,theta_1_true_grad)
print('The average value of stochastic gradient of theta_0, theta_1 for all points in the dataset:',t0_avg_grad,t1_avg_grad)

The True gradient values of theta_0, theta_1: -5.447053909301758 -1.1946825981140137
The average value of stochastic gradient of theta_0, theta_1 for all points in the dataset: -5.447053959220648 -1.194682613387704


In [5]:
print('The difference between avg of stochastic and true grad is',abs(theta_0_true_grad-t0_avg_grad),abs(theta_1_true_grad-t1_avg_grad))

The difference between avg of stochastic and true grad is 4.9918890354661016e-08 1.527369031251169e-08


In [11]:
def full_batch_grad(x,y,lr = 0.01,epsilon = 0.001,epochs = 5000):
    theta_0 = torch.tensor(0.0,requires_grad=True)
    theta_1 = torch.tensor(0.0,requires_grad=True)
    t0_hist = [theta_0.item()]
    t1_hist = [theta_1.item()]
    losses = []
    epoch = 1
    loss = torch.tensor(float('inf'))
    if epochs is not None:
        while loss.item()>=epsilon:
            y_hat = reg(x,theta_0,theta_1)
            loss = mse(y_hat,y)
            losses.append(loss.item())
            loss.backward()
            with torch.no_grad():
                theta_0.data = theta_0-lr*theta_0.grad
                theta_1.data = theta_1-lr*theta_1.grad

            t0_hist.append(theta_0.item())
            t1_hist.append(theta_1.item())
            # print(f'Epoch:{epoch}--> Loss:{loss.item()}')
            theta_0.grad.zero_()
            theta_1.grad.zero_()
            epoch += 1
            if epoch>=epochs:
                break
            if epoch>1000 and epoch%100 == 0:
                if losses[-5:].count(losses[-1])== 5:
                    break
    else:
        while loss.item()>=epsilon:
            y_hat = reg(x,theta_0,theta_1)
            loss = mse(y_hat,y)
            losses.append(loss)
            loss.backward()
            with torch.no_grad():
                theta_0.data = theta_0-lr*theta_0.grad
                theta_1.data = theta_1-lr*theta_1.grad

            t0_hist.append(theta_0.item())
            t1_hist.append(theta_1.item())
            # print(f'Epoch:{epoch}--> Loss:{loss.item()}')
            theta_0.grad.zero_()
            theta_1.grad.zero_()
            epoch += 1
            if epoch>1000 and epoch%100 == 0:
                if losses[-5:].count(losses[-1])== 5:
                    break
                
    print(f'Full Batch Gradient Descent---->No of Epochs Caliculated:{epoch} Final loss:{losses[-1]}')
    return losses, theta_0.grad.item(), theta_1.grad.item(),t0_hist,t1_hist
def mini_batch_grad(x,y,lr = 0.01,batch_size = 2,epsilon = 0.001, epochs = 1500):
    num_samples = x.size(0)
    theta_0 = torch.tensor(0.0,requires_grad=True)
    theta_1 = torch.tensor(0.0,requires_grad=True)
    t0_hist = [theta_0.item()]
    t1_hist = [theta_1.item()]
    losses = []
    avg_epoch_loss = float('inf')
    num_samples_per_batch = num_samples//batch_size
    epoch = 1
    if epochs is not None:
        while avg_epoch_loss >= epsilon:
            batch = torch.randperm(num_samples)
            epoch_loss = 0
            for i in range(0,num_samples, batch_size):
                indice = batch[i:i+batch_size]
                mini_x,mini_y = x[indice],y[indice]
                mini_y_hat = reg(mini_x, theta_0,theta_1)
                loss = mse(mini_y_hat,mini_y)
                epoch_loss += loss.item()
                loss.backward()
                with torch.no_grad():
                    theta_0.data -= lr*theta_0.grad
                    theta_1.data -= lr*theta_1.grad

                t0_hist.append(theta_0.item())
                t1_hist.append(theta_1.item())
                theta_0.grad.zero_()
                theta_1.grad.zero_()
            avg_epoch_loss = epoch_loss/num_samples_per_batch
            losses.append(avg_epoch_loss)
            # print(f'Epoch:{epoch}--> Avg Epoch Loss:{avg_epoch_loss}')
            epoch += 1
            if epoch >=epochs:
                break
            if epoch>1000 and epoch%100 == 0:
                if losses[-5:].count(losses[-1])== 5:
                    break
            
    else:
        while avg_epoch_loss >= epsilon:
            batch = torch.randperm(num_samples)
            epoch_loss = 0
            for i in range(0,num_samples, batch_size):
                indice = batch[i:i+batch_size]
                mini_x,mini_y = x[indice],y[indice]
                mini_y_hat = reg(mini_x, theta_0,theta_1)
                loss = mse(mini_y_hat,mini_y)
                epoch_loss += loss.item()
                loss.backward()
                with torch.no_grad():
                    theta_0.data -= lr*theta_0.grad
                    theta_1.data -= lr*theta_1.grad
                theta_0.grad.zero_()
                theta_1.grad.zero_()

                t0_hist.append(theta_0.item())
                t1_hist.append(theta_1.item())
            avg_epoch_loss = epoch_loss/num_samples_per_batch
            losses.append(avg_epoch_loss)
            # print(f'Epoch:{epoch}--> Avg Epoch Loss:{avg_epoch_loss}')
            epoch += 1
            if epoch>1000 and epoch%100 == 0:
                if losses[-5:].count(losses[-1])== 5:
                    break

    print(f'Mini Batch Gradient Descent---->No of Epochs Caliculated:{epoch} Final loss:{losses[-1]}')
    return losses,theta_0.grad.item(),theta_1.grad.item(),t0_hist,t1_hist
    
def stochastic_grad(x, y, lr=0.01, epsilon=0.001, epochs= 10000):
    num_samples = x.size(0)
    theta_0 = torch.tensor(0.0,requires_grad=True)
    theta_1 = torch.tensor(0.0,requires_grad=True)
    t0_hist = [theta_0.item()]
    t1_hist = [theta_1.item()]
    losses = []
    avg_epoch_loss = float('inf')
    epoch = 1
    if epochs is not None:
        while avg_epoch_loss>=epsilon:
            epoch_loss = 0
            for i in range(num_samples):
                y_hat = reg(x[i], theta_0,theta_1)
                loss =mse(y_hat, y[i])
                epoch_loss += loss.item()
                loss.backward()
                with torch.no_grad():
                    theta_0.data -= lr*theta_0.grad
                    theta_1.data -= lr*theta_1.grad

                t0_hist.append(theta_0.item())
                t1_hist.append(theta_1.item())
                theta_0.grad.zero_()
                theta_1.grad.zero_()
            avg_epoch_loss = epoch_loss/num_samples
            losses.append(avg_epoch_loss)
            print(f'Epoch:{epoch}-->Avg Epoch Loss:{avg_epoch_loss}')
            epoch += 1
            if epoch>= epochs:
                break
            if epoch>1000 and epoch%100 == 0:
                if losses[-5:].count(losses[-1])== 5:
                    break
    else:
        while avg_epoch_loss>=epsilon:
            epoch_loss = 0
            for i in range(num_samples):
                y_hat = reg(x, theta_0,theta_1)
                loss =mse(y_hat, y)
                epoch_loss += loss.item()
                loss.backward()
                with torch.no_grad():
                    theta_0.data -= lr*theta_0.grad
                    theta_1.data -= lr*theta_1.grad
                t0_hist.append(theta_0.item())
                t1_hist.append(theta_1.item())
                theta_0.grad.zero_()
                theta_1.grad.zero_()
            avg_epoch_loss = epoch_loss/num_samples
            # print(f'Epoch:{epoch}-->Avg Epoch Loss:{avg_epoch_loss}')
            losses.append(avg_epoch_loss)
            epoch += 1
            if epoch>500 and epoch%100 == 0:
                if losses[-5:].count(losses[-1])== 5:
                    break
    
    print(f'Stochastic Gradient Descent---->No of Epochs Caliculated:{epoch} Final loss:{losses[-1]}')
    return losses, theta_0.grad.item(), theta_1.grad.item(),t0_hist,t1_hist

In [12]:
losses_full, theta_0_full, theta_1_full, t0_hist_full, t1_hist_full = full_batch_grad(x1_torch, y_torch)
losses_mini, theta_0_mini, theta_1_mini, t0_hist_mini, t1_hist_mini = mini_batch_grad(x1_torch,y_torch)
losses_stoc, theta_0_stoc, theta_1_stoc, t0_hist_stoc, t1_hist_stoc = stochastic_grad(x1_torch,y_torch)

Full Batch Gradient Descent---->No of Epochs Caliculated:2000 Final loss:0.5957541567401083
Mini Batch Gradient Descent---->No of Epochs Caliculated:1500 Final loss:0.603239492889911
Epoch:1-->Avg Epoch Loss:9.665574760781134
Epoch:2-->Avg Epoch Loss:3.359490477893722
Epoch:3-->Avg Epoch Loss:1.7423228651652571
Epoch:4-->Avg Epoch Loss:1.1924672300909003
Epoch:5-->Avg Epoch Loss:0.9430930451065574
Epoch:6-->Avg Epoch Loss:0.8081935674541008
Epoch:7-->Avg Epoch Loss:0.7295188373196498
Epoch:8-->Avg Epoch Loss:0.6824056090150223
Epoch:9-->Avg Epoch Loss:0.6539661834577192
Epoch:10-->Avg Epoch Loss:0.6367739230524919
Epoch:11-->Avg Epoch Loss:0.6263919907100048
Epoch:12-->Avg Epoch Loss:0.6201368882920935
Epoch:13-->Avg Epoch Loss:0.6163802099113462
Epoch:14-->Avg Epoch Loss:0.6141340064571434
Epoch:15-->Avg Epoch Loss:0.6127984197550671
Epoch:16-->Avg Epoch Loss:0.612010357741376
Epoch:17-->Avg Epoch Loss:0.6115499643855358
Epoch:18-->Avg Epoch Loss:0.6112850830826757
Epoch:19-->Avg Epoc